# Wordcount in Spark

### Setup

Let's setup Spark on the Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4 MB 67 kB/s 
     |████████████████████████████████| 198 kB 35.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=31eded5de79abe2117498cb549186935db4396bfbeca160249f369b05f7fd2bf
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

Lets authenticate a Google Drive client to download the file we will be processing in on Spark job.

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

By executed the cells above, we can able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Wordcount

After running the setup stage successfully, I am ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

I am going to write a Spark application which outputs the number of words that start with each letter. This means that for every letter I want to count the total number of (non-unique) words that start with a specific letter. In my implementation, I am **ignoring the letter case**, i.e., consider all words as lower case. Also, I am ignoring all the words **starting** with a non-alphabetic character.

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [5]:
!head -5 pg100.txt

﻿The Project Gutenberg EBook of The Complete Works of William Shakespeare, by
William Shakespeare

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or


In [6]:
pg100 = sc.textFile('pg100.txt')

In [7]:
counts = pg100.flatMap(lambda line: line.split(" ")) \
              .filter(lambda word: len(word) > 0)  \
              .filter(lambda word: ord(word.lower()[0]) in range(ord('a'), ord('z')+1)) \
              .map(lambda word: (word.lower()[0], 1)) \
              .reduceByKey(lambda a, b: a + b)

In [8]:
counts.collect()

[('p', 27759),
 ('g', 20782),
 ('c', 34567),
 ('s', 65705),
 ('b', 45455),
 ('i', 62167),
 ('r', 14265),
 ('y', 25855),
 ('l', 29569),
 ('d', 29713),
 ('j', 3339),
 ('h', 60563),
 ('t', 123602),
 ('e', 18697),
 ('o', 43494),
 ('w', 59597),
 ('f', 36814),
 ('u', 9170),
 ('a', 84836),
 ('n', 26759),
 ('m', 55676),
 ('v', 5728),
 ('k', 9418),
 ('q', 2377),
 ('z', 71),
 ('x', 14)]

In [9]:
counts.saveAsTextFile("char_count.txt")

In [10]:
sc.stop()